In [ ]:
!pip install --quiet flair

In [ ]:
!pip install --quiet aiohttp==3.9.2

In [ ]:
!pip install --quiet botocore==1.34.70

In [ ]:
!pip install --upgrade --quiet boto3

In [2]:
from flair.trainers import ModelTrainer

In [1]:
import shutil
import re

from flair.data import Corpus
from flair.data import Sentence

from flair.datasets import ColumnCorpus
from flair.embeddings import WordEmbeddings, StackedEmbeddings, FlairEmbeddings
from flair.models import SequenceTagger
from flair.embeddings import TransformerWordEmbeddings

from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR

2024-08-11 10:21:34.067594: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 10:21:34.067718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 10:21:34.205052: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [13]:
columns = {0: 'text', 1: 'ner'}

corpus: Corpus = ColumnCorpus('/kaggle/input/wnut-and-idrisi/',columns,
                              train_file='TRAIN_final (1).txt',
                              dev_file='TEST_final (1).txt',
                              test_file='TEST_final (1).txt')

2024-08-11 10:44:31,564 Reading data from /kaggle/input/wnut-and-idrisi
2024-08-11 10:44:31,565 Train: /kaggle/input/wnut-and-idrisi/TRAIN_final (1).txt
2024-08-11 10:44:31,566 Dev: /kaggle/input/wnut-and-idrisi/TEST_final (1).txt
2024-08-11 10:44:31,568 Test: /kaggle/input/wnut-and-idrisi/TEST_final (1).txt


In [14]:
tag_type = 'ner'

tag_dictionary = corpus.make_label_dictionary(label_type=tag_type)

embeddings = TransformerWordEmbeddings(
    model='tner/deberta-v3-large-ontonotes5',
    layers="-1",
    subtoken_pooling="first",
    fine_tune=True,
    use_context=True,
)

2024-08-11 10:44:51,305 Computing label dictionary. Progress:


0it [00:00, ?it/s]
16786it [00:00, 37359.80it/s]

2024-08-11 10:44:51,762 Dictionary created for label 'ner' with 1 values: LOC (seen 15639 times)


In [15]:
tagger = SequenceTagger(
    hidden_size=1024,
    embeddings=embeddings,
    tag_dictionary=tag_dictionary,
    tag_type='ner',
    use_crf=False,
    use_rnn=False,
    reproject_embeddings=True,gthmllhy,,mki9knhubcbcbhfg
)

2024-08-11 10:48:01,980 SequenceTagger predicts: Dictionary with 5 tags: O, S-LOC, B-LOC, E-LOC, I-LOC


In [17]:
trainer = ModelTrainer(tagger, corpus)

trainer.train('model',
              learning_rate=5.0e-6,
              mini_batch_size=8,
              mini_batch_chunk_size=1,
              max_epochs=3,
              optimizer=AdamW,
              #scheduler=OneCycleLR,
              embeddings_storage_mode='none',
              weight_decay=0.
              )

2024-08-11 10:49:02,427 ----------------------------------------------------------------------------------------------------
2024-08-11 10:49:02,432 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): DebertaV2Model(
      (embeddings): DebertaV2Embeddings(
        (word_embeddings): Embedding(128101, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
        (dropout): StableDropout()
      )
      (encoder): DebertaV2Encoder(
        (layer): ModuleList(
          (0-23): 24 x DebertaV2Layer(
            (attention): DebertaV2Attention(
              (self): DisentangledSelfAttention(
                (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
                (pos_dropout): StableDropout()
                (dropout): StableDropout()
    

100%|██████████| 33/33 [02:04<00:00,  3.78s/it]

2024-08-11 11:47:11,143 DEV : loss 0.03994711488485336 - f1-score (micro avg)  0.896
2024-08-11 11:47:11,200  - 0 epochs without improvement
2024-08-11 11:47:11,201 ----------------------------------------------------------------------------------------------------
2024-08-11 11:47:11,202 learning rate too small - quitting training!
2024-08-11 11:47:11,203 ----------------------------------------------------------------------------------------------------
2024-08-11 11:47:11,204 Saving model ...


2024-08-11 11:47:13,449 Done.
2024-08-11 11:47:13,450 ----------------------------------------------------------------------------------------------------
2024-08-11 11:47:13,455 Testing using last state of model ...


100%|██████████| 33/33 [02:04<00:00,  3.78s/it]

2024-08-11 11:49:18,322 
Results:
- F-score (micro) 0.896
- F-score (macro) 0.896
- Accuracy 0.8116

By class:
              precision    recall  f1-score   support

         LOC     0.8780    0.9148    0.8960      2194

   micro avg     0.8780    0.9148    0.8960      2194
   macro avg     0.8780    0.9148    0.8960      2194
weighted avg     0.8780    0.9148    0.8960      2194

2024-08-11 11:49:18,323 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8959821428571428}

In [18]:
model = SequenceTagger.load('/kaggle/working/model/final-model.pt')
sentence = Sentence('I love Southern California and texas')
model.predict(sentence)
print(sentence)

2024-08-11 12:18:23,787 SequenceTagger predicts: Dictionary with 5 tags: O, S-LOC, B-LOC, E-LOC, I-LOC
Sentence[6]: "I love Southern California and texas" → ["California"/LOC, "texas"/LOC]


In [19]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/model{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

download_file('/kaggle/working/model', 'out')
egtr

/kaggle/working/out.zip

In [21]:
import json

directory_path = "/kaggle/input/idrisi-gold-random-json/IDRISI Gold Random JSON"

train_data = []
dev_data = []
test_unlabeled_data = []

for folder in os.listdir(directory_path):
    folder_path = os.path.join(directory_path, folder)

    if os.path.isdir(folder_path):
        train_jsonl_path = os.path.join(folder_path, "train.jsonl")
        dev_jsonl_path = os.path.join(folder_path, "dev.jsonl")
        test_unlabeled_jsonl_path = os.path.join(folder_path, "test_unlabeled.jsonl")

        with open(train_jsonl_path, "r", encoding="utf-8") as train_file:
            for line in train_file:
                train_data.append(json.loads(line.strip()))

        with open(dev_jsonl_path, "r", encoding="utf-8") as dev_file:
            for line in dev_file:
                dev_data.append(json.loads(line.strip()))

        with open(test_unlabeled_jsonl_path, "r", encoding="utf-8") as test_unlabeled_file:
            for line in test_unlabeled_file:
                test_unlabeled_data.append(json.loads(line.strip()))


combined_train_data = {}
for i, item in enumerate(train_data, start=1):
    combined_train_data[f"entry_{i}"] = item


combined_dev_data = {}
for i, item in enumerate(dev_data, start=1):
    combined_dev_data[f"entry_{i}"] = item

combined_test_unlabeled_data = {}
for i, item in enumerate(test_unlabeled_data, start=1):
    combined_test_unlabeled_data[f"entry_{i}"] = item

In [ ]:
cntr = 0

for entry_key, entry_data in combined_dev_data.items():
    print("Sample",cntr)
    cntr+=1
    tweet_id = entry_data['tweet_id']
    text = entry_data['text']
    sentence = Sentence(text)
    # predict NER tags
    model.predict(sentence)
    pred = []

    for entity in sentence.get_spans('ner'):
        if (entity.tag=='LOC'):
            pred.append({ 'entity': entity.tag, 'word': entity.text, 'start' : entity.start_position, 'end' : entity.end_position})

    print(text)
    print("True Locations")
    for lo in entry_data['location_mentions']:
        print(lo)

    print("Predicted Locations")
    print(pred,'\n\n\n')

In [38]:
test_df = pd.read_csv("/kaggle/input/microsoft-lmr-dataset/Test.csv")
test_df.head()

,tweet_id,text
0,ID_1001154804658286592,What is happening to the infrastructure in New...
1,ID_1001155505459486720,SOLDER MISSING IN FLOOD.. PRAY FOR EDDISON HER...
2,ID_1001155756371136512,RT @TIME: Police searching for missing person ...
3,ID_1001159445194399744,Flash Flood Tears Through Maryland Town For Se...
4,ID_1001164907587538944,Ellicott City #FLOODING Pictures: Maryland Gov...


In [ ]:
list1 = []
list2 = []
cntr = 0

for entry_key, entry_data in combined_test_unlabeled_data.items():
    print("Sample",cntr)
    cntr+=1
    tweet_id = entry_data['tweet_id']
    text = entry_data['text']
    sentence = Sentence(text)
    model.predict(sentence)
    pred = []

    for entity in sentence.get_spans('ner'):
        if (entity.tag=='LOC'):
            pred.append({ 'entity': entity.tag, 'word': entity.text, 'start' : entity.start_position, 'end' : entity.end_position})

    print(text)
    print("Predicted Locations")
    print(pred,'\n\n\n')

    entity_count = 1

    for entity in pred:
        entity_group = entity['entity']
        start = entity['start']
        end = entity['end']

        # Generate the ID strings and append them to list1
        id_str_start = f'ID_{tweet_id}_loc{entity_count}_start'
        id_str_end = f'ID_{tweet_id}_loc{entity_count}_end'
        list1.append(id_str_start)
        list1.append(id_str_end)
        # Append the start and end values to list2
        list2.append(start)
        list2.append(end)
        entity_count += 1

    if (entity_count<=17):
        for i in range(entity_count,18):
            id_str_start = f'ID_{tweet_id}_loc{entity_count}_start'
            id_str_end = f'ID_{tweet_id}_loc{entity_count}_end'
            list1.append(id_str_start)
            list1.append(id_str_end)

            list2.append(0)
            list2.append(0)
            entity_count += 1
# Now list1 contains the ID strings and list2 contains the start and end values

In [39]:
# Assuming model is already defined
# list1 and list2 will store the results
list1 = []
list2 = []

# Initialize counter for samples
cntr = 0

# Iterate through the dataframe
for index, row in test_df.iterrows():
    print("Sample", cntr)
    cntr += 1
    
    # Extract tweet_id and text from the dataframe row
    tweet_id = row['tweet_id']
    text = row['text']
    
    # Process the text with the model
    sentence = Sentence(text)
    model.predict(sentence)
    pred = []

    # Extract location entities (LOC) and their positions
    for entity in sentence.get_spans('ner'):
        if entity.tag == 'LOC':
            pred.append({
                'entity': entity.tag,
                'word': entity.text,
                'start': entity.start_position,
                'end': entity.end_position
            })

    print(text)
    print("Predicted Locations")
    print(pred, '\n\n\n')

    # Initialize entity count
    entity_count = 1

    for entity in pred:
        start = entity['start']
        end = entity['end']

        # Generate ID strings and append them to list1
        id_str_start = f'ID_{tweet_id}_loc{entity_count}_start'
        id_str_end = f'ID_{tweet_id}_loc{entity_count}_end'
        list1.append(id_str_start)
        list1.append(id_str_end)

        # Append the start and end values to list2
        list2.append(start)
        list2.append(end)
        entity_count += 1

    # Fill up to 17 entities with zeros if less than 17 found
    if entity_count <= 17:
        for i in range(entity_count, 18):
            id_str_start = f'ID_{tweet_id}_loc{entity_count}_start'
            id_str_end = f'ID_{tweet_id}_loc{entity_count}_end'
            list1.append(id_str_start)
            list1.append(id_str_end)

            list2.append(0)
            list2.append(0)
            entity_count += 1

# Now list1 contains the ID strings and list2 contains the start and end values


Sample 0
What is happening to the infrastructure in New England? It isnt global warming, its misappropriated funds being abused that shouldve been used maintaining their infrastructure that couldve protected them from floods! Like New Orleans. Their mayor went to ὄ7#Maryland #floods
Predicted Locations
[{'entity': 'LOC', 'word': 'New England', 'start': 43, 'end': 54}, {'entity': 'LOC', 'word': 'New Orleans', 'start': 222, 'end': 233}, {'entity': 'LOC', 'word': 'Maryland', 'start': 258, 'end': 266}] 



Sample 1
SOLDER MISSING IN FLOOD.. PRAY FOR EDDISON HERMOND! PRAY FOR ELLICOTT CITY, MARYLAND! #PrayForEddisonHermond #PrayForEllicottCity
Predicted Locations
[{'entity': 'LOC', 'word': 'ELLICOTT CITY', 'start': 61, 'end': 74}, {'entity': 'LOC', 'word': 'MARYLAND', 'start': 76, 'end': 84}] 



Sample 2
RT @TIME: Police searching for missing person after devastating 1,000-year flood in Ellicott City, Maryland
Predicted Locations
[{'entity': 'LOC', 'word': 'Ellicott City', 'start': 84, 'en

In [46]:
import pandas as pd

data = {'Tweet_ID': list1, 'Index': list2}
df = pd.DataFrame(data)
df.fillna(" ")

csv_filename = 'final_sub.csv'

df.to_csv(csv_filename, index=False)

print(f'Data written to {csv_filename}.')

Data written to final_sub.csv.


In [42]:
print(df.head())

                               Tweet_ID  Index
0  ID_ID_1001154804658286592_loc1_start     43
1    ID_ID_1001154804658286592_loc1_end     54
2  ID_ID_1001154804658286592_loc2_start    222
3    ID_ID_1001154804658286592_loc2_end    233
4  ID_ID_1001154804658286592_loc3_start    258


In [27]:
if "ID_1001155505459486729" in list1:
    print("Yes")
else:
    print("No")

No
